In [1]:
import os
os.chdir('..')
from bilevel.build_all_models import *
from bilevel.utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%%time
df_adult = pd.read_csv('./data/adult_reconstruction.csv')

A_t_age = pd.concat([df_adult['age'] <= 35,  # young
        (df_adult['age'] > 35) & (df_adult['age'] <= 50), # middle
        df_adult['age'] > 50], # old
             axis = 1, keys = ['young', 'middle', 'old'])

A_t_edu = pd.concat([df_adult['education-num'] <= 9, df_adult['education-num'] >= 10], axis = 1, 
                keys = ['HighSchool&less', 'College&more'])

A_t_sex = pd.concat([df_adult['sex'] == 'Male', df_adult['sex'] == 'Female'], axis = 1, keys = ['Male', 'Female'])

A_t_race = pd.concat([df_adult['race'] == 'White', df_adult['race'] == 'Asian-Pac-Islander',
                      df_adult['race'] == 'Amer-Indian-Eskimo', df_adult['race'] == 'Other',
                      df_adult['race'] == 'Black'], axis = 1, keys = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'])
A_t_adult_groups = pd.concat([A_t_age, A_t_edu, A_t_sex, A_t_race], axis=1)
A_t_adult_groups['always_on'] = True
A_t_adult_groups *= 1
A_t_adult_groups

CPU times: user 79.1 ms, sys: 18.9 ms, total: 98 ms
Wall time: 95.9 ms


,young,middle,old,HighSchool&less,College&more,Male,Female,White,Asian-Pac-Islander,Amer-Indian-Eskimo,Other,Black,always_on
0,0,1,0,0,1,0,1,1,0,0,0,0,1
1,1,0,0,0,1,1,0,1,0,0,0,0,1
2,1,0,0,1,0,1,0,1,0,0,0,0,1
3,0,0,1,1,0,1,0,0,1,0,0,0,1
4,1,0,0,0,1,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49526,1,0,0,0,1,1,0,1,0,0,0,0,1
49527,0,1,0,0,1,1,0,0,1,0,0,0,1
49528,1,0,0,0,1,1,0,1,0,0,0,0,1
49529,1,0,0,0,1,0,1,1,0,0,0,0,1


In [4]:
added_indicators = ['young', 'middle', 'old', 'HighSchool&less', 'College&more'] # adding group indicators to dataframe, so baseline knows the groupstoo
numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income'] 
cat_feat =  ['workclass', 'marital-status', 'relationship', 'native-country', 'occupation', 'race', 'sex']

df_adult.drop(['education'], axis = 1, inplace=True)
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh[added_indicators] = A_t_adult_groups[added_indicators]
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,race_Black,race_Other,race_White,sex_Female,sex_Male,young,middle,old,HighSchool&less,College&more
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0,1,0,0,1
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1,0,0,0,1
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1,0,0,1,0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0,0,1,1,0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1,0,0,0,1


In [ ]:
%%time
from bilevel.Groupwise_seedruns import BuildGroupwise_diffseeds
ds_obj = BuildGroupwise_diffseeds(df_adult_mm_oh, 'income', A_t_adult_groups)
ds_obj.build_all_seeds()
ds_obj.build_df_res()
ds_obj.build_regret_curve()

100%|██████████| 49531/49531 [00:35<00:00, 1399.87it/s]


In [ ]:
from bilevel.Groupwise_seedruns import plot_regret_curve_with_std # with clipped y_vals of the least squares predictor
plot_regret_curve_with_std(ds_obj, './plots/adult')

In [ ]:
from bilevel.Groupwise_seedruns import get_end_regret_gw_df
df_regend_adult = get_end_regret_gw_df(ds_obj)

In [ ]:
df_regend_adult

In [ ]:
df_regend_adult['mean_hindsight'].mean(axis=0), (df_regend_adult['mean_regend_Base'] - df_regend_adult['mean_regend_Anh']).mean(axis=0) # rough values for cumloss, regret difference base-our alg

In [ ]:
df_regend_adult.to_csv('./tables/adultincome_res.csv')